In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# Import

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data.sampler import BatchSampler

In [ ]:
import  json

In [ ]:
import math

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:

import os
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
import random
import time
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [ ]:
import csv
import json
from sklearn import *
import json, sys, regex
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from numpy.random import RandomState
import pandas as pd
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

In [ ]:
pip install wget

In [ ]:
import os
import wget
import gzip
import shutil

In [ ]:
!pip install transformers

In [ ]:
from transformers import AdamW

In [ ]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

In [ ]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [ ]:
!mkdir -p  isarcastic
!cp "/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_train.csv"  ./isarcastic/isarcastic_Ar_train.csv
!cp "/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_test1.csv"  ./isarcastic/isarcastic_Ar_test1.csv
!cp "/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_test2.csv"  ./isarcastic/isarcastic_Ar_test2.csv

In [ ]:
!cp "/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_train.csv"  ./isarcastic/isarcastic_Ar_train.cs


In [ ]:
!cp "/gdrive/MyDrive/Master/Dataset/TestingData/task_A_AR_test.csv"  ./isarcastic/task_A_AR_test.csv

In [ ]:
test_file2 = './isarcastic/task_A_AR_test.csv'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

In [ ]:
df = pd.read_csv('./isarcastic/isarcastic_Ar_train.csv', sep=",")
df.head(5)

In [ ]:
df = pd.read_csv('isarcastic/isarcastic_Ar_test1.csv', sep=",")
df.head(5)

In [ ]:

config={"task_name": "isarcastic_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "checkpoint_dir": "./",
             "train_file": "isarcastic/isarcastic_Ar_train.csv", #train file path
             "dev_file": "isarcastic/isarcastic_Ar_test1.csv", #dev file path or test file path
             "test_file": "isarcastic/isarcastic_Ar_test2.csv", #dev file path or test file path
        

             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
            #  "fine_tuned_model_path" : "/gdrive/MyDrive/Master/Dataset/isarcasticisarcastic_MARBERT_bert_ckpt/model_5",
             "epochs": 1, #number of epochs
             "content_col": "tweet", #text column
             "label_col": "sarcastic", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 64, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
#---------------------------------------
print ("[INFO] step (1) load train_test config file")
task_name = config["task_name"]
content_col = config["content_col"]
label_col = config["label_col"]
train_file = config["data_dir"]+config["train_file"]
dev_file = config["data_dir"]+config["dev_file"]
test_file = config["data_dir"]+config["test_file"]
sortby = config["sortby"]
max_seq_length= int(config["max_seq_length"])
batch_size = int(config["batch_size"])
lr_var = float(config["lr"])
model_path = config['pretrained_model_path']
num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
global label2idx_file
label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
#-------------------------------------------------------
print ("[INFO] step (2) convert labels2index")

In [ ]:

def create_label2ind_file(file, label_col):
	labels_json={}
	#load train_dev_test file
	df = pd.read_csv(file, sep=",")
	df.head(5)
	#get labels and sort it A-Z
	labels = df[label_col].unique()
	labels.sort()
	#convert labels to indexes
	for idx in range(0, len(labels)):
		labels_json[labels[idx]]=idx


In [ ]:
create_label2ind_file(train_file, label_col)
print (label2idx_file)

In [ ]:
#---------------------------------------------------------
print ("[INFO] step (3) check checkpoit directory and report file")
ckpt_dir = config["checkpoint_dir"]+task_name+"_bert_ckpt/"
report = ckpt_dir+task_name+"_report.tsv"
sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
if not os.path.exists(ckpt_dir):
  os.mkdir(ckpt_dir)

In [ ]:
#-------------------------------------------------------
print ("[INFO] step (4) load label to number dictionary")

In [ ]:
x =  '{ "0":0, "1":1}'
y = json.loads(x)

In [ ]:
lab2ind = json.loads(x)

In [ ]:
print ("[INFO] train_file", train_file)
print ("[INFO] dev_file", dev_file)
print ("[INFO] num_epochs", num_epochs)
print ("[INFO] model_path", model_path)
print ("max_seq_length", max_seq_length, "batch_size", batch_size)
#-------------------------------------------------------

In [ ]:
def data_prepare(file_path, lab2ind, content_col, label_col, MAX_LEN):
  df = pd.read_csv(file_path, delimiter=',', header=0)
  df = df[df[content_col].notnull()]
  df = df[df[label_col].notnull()]
  print("Data size ", df.shape)
  sentences = df[content_col].values
  print ("The first sentence:")
  print (sentences[0])
  labels = df[label_col].values
  print(labels)
  labels = [lab2ind[ str(i)] for i in labels]
  return sentences,labels

In [ ]:
def data_prepare2(file_path, lab2ind, content_col, MAX_LEN):
  df = pd.read_csv(file_path, delimiter=',', header=0)
  df = df[df[content_col].notnull()]
  print("Data size ", df.shape)
  sentences = df[content_col].values
  print ("The first sentence:")
  print (sentences[0])

  return sentences

In [ ]:

def data_prepare3(file_path, lab2ind, content_col_0, content_col_1, label_col, MAX_LEN):
  df = pd.read_csv(file_path, delimiter=',', header=0)
  print("df before notnull", df)
  df = df[df[content_col_0].notnull()]
  df = df[df[content_col_1].notnull()]
  print("df after notnull", df)
  print("Data size ", df.shape)
  sentences_0 = df[content_col_0].values
  sentences_1 = df[content_col_1].values
  sentences_0 = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences_0]
  sentences_1 = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences_1]
  print ("The first sentence:")
  print (sentences_0[0])
  print (sentences_1[0])
  
  return sentences_0,sentences_1


In [ ]:
test_file3

In [ ]:
test_inputs03,test_inputs13 

In [ ]:
test_inputs03,test_inputs13 = data_prepare3('isarcastic/task_C_AR_test.csv', lab2ind, "text_0", "text_1", label_col,max_seq_length)


In [ ]:
test

In [ ]:
test_inputs2 = data_prepare2(test_file2, lab2ind,'text', max_seq_length)

In [ ]:
print(len(test_inputs2))

In [ ]:
train_inputs, train_labels = data_prepare(train_file, lab2ind,content_col, label_col, max_seq_length)

In [ ]:
validation_inputs, validation_labels = data_prepare(dev_file, lab2ind, content_col, label_col,max_seq_length)

In [ ]:
test_inputs, test_labels = data_prepare(test_file, lab2ind, content_col, label_col,max_seq_length)

In [ ]:
%%time
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz"

In [ ]:
FILE = "fastText"

In [ ]:
if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    with gzip.open( 'fastText/cc.ar.300.vec.gz', 'rb') as f_in:
            with open( 'fastText/cc.ar.300.vec', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [ ]:
bert = AutoModel.from_pretrained('UBC-NLP/MARBERT')
tokenizer = BertTokenizerFast.from_pretrained('UBC-NLP/MARBERT')

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
df = pd.read_csv("/gdrive/MyDrive/Master/Dataset/TrainingData/train.Ar.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df['sarcastic'].value_counts(normalize = True)

In [ ]:
df_train = pd.read_csv("/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_train.csv")
df_train.head()

In [ ]:
df_test3 = pd.read_csv(test_file3)
df_test3.head()

In [ ]:
df_test = pd.read_csv("/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_test2.csv")
df_test.head()

In [ ]:
df_test2 = pd.read_csv("./isarcastic/task_A_AR_test.csv")
df_test2.head()

In [ ]:
df_val = pd.read_csv("/gdrive/MyDrive/Master/Dataset/isarcastic/isarcastic_Ar_train_test/isarcastic_Ar_test1.csv")
df_val.head()

# Split train dataset into train, validation and test sets

In [ ]:
text_temp = df['tweet']

In [ ]:
train_text, train_labels = df_train['tweet'], df_train['sarcastic']

In [ ]:
test_text, test_labels = df_test['tweet'], df_test['sarcastic']

In [ ]:
test_text2 = df_test2['text']

In [ ]:
test_text03, test_text13 = df_test3['text_0'], df_test3['text_1']

In [ ]:
val_text, val_labels = df_val['tweet'], df_val['sarcastic']


In [ ]:
len(train_text), len(val_text), len(test_text), len(val_labels), len(test_labels)

In [ ]:
len(train_text), len(val_text), len(test_text), len(val_labels), len(test_labels)

In [ ]:
train_text

In [ ]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]

# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [ ]:
# output
print(sent_id)

# Length of train

In [ ]:
seq_len = [len(i.split()) for i in text_temp]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
max(seq_len)

In [ ]:
max_seq_len = 64

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
tokens_test2 = tokenizer.batch_encode_plus(
    test_text2.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
tokens_test03 = tokenizer.batch_encode_plus(
    test_text03.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
tokens_test13 = tokenizer.batch_encode_plus(
    test_text13.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
tokens_test03

In [ ]:
tokens_val

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
test_seq2 = torch.tensor(tokens_test2['input_ids'])
test_mask2 = torch.tensor(tokens_test2['attention_mask'])

In [ ]:
test_seq03 = torch.tensor(tokens_test03['input_ids'])
test_mask03 = torch.tensor(tokens_test03['attention_mask'])

In [ ]:
test_seq13 = torch.tensor(tokens_test13['input_ids'])
test_mask13 = torch.tensor(tokens_test13['attention_mask'])

In [ ]:
len(test_seq13)

# Remove sampler used

In [ ]:
batch_size = 32
train_data = TensorDataset(train_seq, train_mask, train_y)
train_dataloader2 = DataLoader(train_data, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_dataloader2 = DataLoader(val_data, batch_size=batch_size)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_dataloader2 = DataLoader(test_data, batch_size=batch_size)

test_data2 = TensorDataset(test_seq2, test_mask2)
test_dataloader22 = DataLoader(test_data2, batch_size=batch_size)

In [ ]:
test_data03 = TensorDataset(test_seq03, test_mask03)
test_dataloader03 = DataLoader(test_data03, batch_size=batch_size)

In [ ]:
test_data13 = TensorDataset(test_seq13, test_mask13)
test_dataloader13 = DataLoader(test_data13, batch_size=batch_size)

# *build dataloader 1*


In [ ]:
word2idx = {}
# Add <pad> and <unk> tokens to the vocabulary
word2idx['<pad>'] = 0
word2idx['<unk>'] = 1


# Building our vocab from the corpus starting from index 2
idx = 2
# max_len = 0

In [ ]:
def tokenize(texts, idx,max_len):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """
    
    tokenized_texts = []
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1


    return tokenized_texts, idx,max_len

def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [ ]:
def load_pretrained_vectors(word2idx, fname):
    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [ ]:
max_len =64

In [ ]:
print("Tokenizing...\n")
tokenized_texts_train,idx,max_len = tokenize(train_inputs, idx,max_len)

In [ ]:
idx,max_len 

In [ ]:
input_ids_train = encode(tokenized_texts_train, word2idx, max_len)

In [ ]:
tokenized_texts_val,idx,max_len = tokenize(validation_inputs, idx,max_len)

In [ ]:
input_ids_val = encode(tokenized_texts_val, word2idx, max_len)

In [ ]:
tokenized_texts_test,idx,max_len = tokenize(test_inputs, idx,max_len)

In [ ]:
input_ids_test = encode(tokenized_texts_test, word2idx, max_len)

In [ ]:
tokenized_texts_test03,idx,max_len = tokenize(test_inputs03, idx,max_len)

In [ ]:
input_ids_test03 = encode(tokenized_texts_test03, word2idx, max_len)

In [ ]:
tokenized_texts_test13,idx,max_len = tokenize(test_inputs13, idx,max_len)

In [ ]:
input_ids_test13 = encode(tokenized_texts_test13, word2idx, max_len)

In [ ]:
tokenized_texts_test2,idx,max_len = tokenize(test_inputs2, idx,max_len)

In [ ]:
tokenized_texts_test2

In [ ]:
input_ids_test2 = encode(tokenized_texts_test2, word2idx, max_len)

In [ ]:
# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "fastText/cc.ar.300.vec")
embeddings = torch.tensor(embeddings)

In [ ]:
14375 / 16263 *100.0

In [ ]:
test_inputs

In [ ]:
def data_loader(train_inputs, val_inputs, test_inputs, train_labels, val_labels, test_labels,test_inputs2,  test_inputs03,test_inputs13
               , batch_size=batch_size):

    # Convert data type to torch.Tensor
    train_inputs, val_inputs,test_inputs, train_labels, val_labels, test_labels,test_inputs2, test_inputs03,test_inputs13 =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs,test_inputs, train_labels, val_labels, test_labels,test_inputs2, test_inputs03,test_inputs13])

    batch_size = batch_size

    train_data = TensorDataset(train_inputs, train_labels)
    train_dataloader = DataLoader(train_data, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_dataloader = DataLoader(val_data, batch_size=batch_size)

    test_data = TensorDataset(test_inputs, test_labels)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    test_data2 = TensorDataset(test_inputs2)
    test_dataloader12 = DataLoader(test_data2, batch_size=batch_size)

    test_data03 = TensorDataset(test_inputs03)
    test_dataloader103 = DataLoader(test_data03, batch_size=batch_size)
    test_data13 = TensorDataset(test_inputs13)
    test_dataloader113 = DataLoader(test_data13, batch_size=batch_size)

    return train_dataloader, val_dataloader, test_dataloader, test_dataloader12, test_dataloader103, test_dataloader113

In [ ]:
train_dataloader, val_dataloader, test_dataloader,test_dataloader12, test_dataloader103, test_dataloader113 = \
data_loader( input_ids_train, input_ids_val, input_ids_test, train_labels, validation_labels, test_labels, input_ids_test2,  test_inputs03,test_inputs13, batch_size=batch_size)

In [ ]:
len(test_dataloader12)

# CNN

**CNN Architecture**

The picture below is the illustration of the CNN architecture that we are going to build with three filter sizes: 2, 3, and 4, each of which has 2 filters.

![](https://github.com/chriskhanhtran/CNN-Sentence-Classification-PyTorch/blob/master/cnn-architecture.JPG?raw=true)

*CNN Architecture (Source: Zhang, 2015)*

```python
# Sample configuration:
filter_sizes = [2, 3, 4]
num_filters = [2, 2, 2]
```

Suppose that we are classifying the sentence "***I like this movie very much!***" ($N = 7$ tokens) and the dimensionality of word vectors is $d=5$. After applying the embedding layer on the input token ids, the sample sentence is presented as a 2D tensor with shape (7, 5) like an image.

$$\mathrm{x_{emb}} \quad \in \mathbb{R}^{7 \times 5}$$

We then use 1-dimesional convolution to extract features from the sentence. In this example, we have 6 filters in total, and each filter has shape $(f_i, d)$ where $f_i$ is the filter size for $i \in \{1,...,6\}$. Each filter will then scan over $\mathrm{x_{emb}}$ and returns a feature map:

$$\mathrm{x_{conv_ i} = Conv1D(x_{emb})} \quad \in \mathbb{R}^{N-f_i+1}$$

Next, we apply the ReLU activation to $\mathrm{x_{conv_{i}}}$ and use max-over-time-pooling to reduce each feature map to a single scalar. Then we concatenate these scalars into the final feature vector which will be fed to a fully connected layer to compute the final scores for our classes (logits).

$$\mathrm{x_{pool_i} = MaxPool(ReLU(x_{conv_i}))} \quad \in \mathbb{R}$$

$$\mathrm{x_{fc} = \texttt{concat}(x_{pool_i})} \quad \in \mathbb{R}^6$$

The idea here is that each filter will capture different semantic signals in the sentence (ie. happiness, humor, politic, anger...) and max-pooling will record only the strongest signal over the sentence. This logic makes sense because humans also perceive the sentiment of a sentence based on its strongest word/signal.

Finally, we use a fully connected layer with the weight matrix $\mathbf{W_{fc}} \in \mathbb{R}^{2 \times 6} $ and dropout to compute $\mathrm{logits}$, which is a vector of length 2 that keeps the scores for 2 classes.

$$\mathrm{logits = Dropout(\mathbf{W_{fc}}x_{fc})}  \in \mathbb{R}^2$$

An in-depth explanation of CNN can be found in this [article](https://cs231n.github.io/convolutional-networks/) and this [video](https://www.youtube.com/watch?v=YRhxdVk_sIs).










### 3.1. Create CNN Model

For simplicity, the model above has very small configurations. The final model we'll use is much bigger but has the same architecture:

|Description         |Values           |
|:------------------:|:---------------:|
|input word vectors  |fastText         |
|embedding size      |300              |
|filter sizes        |(3, 4, 5)        |
|num filters         |(100, 100, 100)  |
|activation          |ReLU             |
|pooling             |1-max pooling    |
|dropout rate        |0.5              |



In [ ]:
class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

### 3.2. Optimizer

To train Deep Learning models, we need to define a loss function and minimize this loss. We'll use back-propagation to compute gradients and use an optimization algorithm (ie. Gradient Descent) to minimize the loss. The original paper used the Adadelta optimizer.

In [ ]:

def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer

### 3.3. Training Loop

For each epoch, the code below will perform a forward step to compute the *Cross Entropy* loss, a backward step to compute gradients and use the optimizer to update weights/parameters. At the end of each epoch, the loss on training data and the accuracy over the validation data will be printed to help us keep track of the model's performance. The code is heavily annotated with detailed explanations.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:


# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0
    best_f1_sarcastic = 0

    # Start training loop
    
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {\
    'Val Acc':^9} | {'f1 sarcastic':^9}| {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy,val_f1_sarcastic, preds_all = evaluate(model, val_dataloader)

            # Track the best accuracy
            # if val_accuracy > best_accuracy:
            #     best_accuracy = val_accuracy
            if val_f1_sarcastic > best_f1_sarcastic:
                best_f1_sarcastic = val_f1_sarcastic
            

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {\
            val_loss:^10.6f} | {val_accuracy:^9.2f} | {val_f1_sarcastic:^9.2f}|{time_elapsed:^9.2f}")
            # print(f"Training complete! Best f1_sarcastic: {best_f1_sarcastic:.2f}%.")
            
    print("\n")
    print(f"Training complete! Best f1_sarcastic: {best_f1_sarcastic:.2f}%.")

In [ ]:
def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []
    preds_all = []
    val_f1_sarcastic = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()
        preds_all.extend(preds)

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
        # print(len(b_labels))
        # print(len(preds))
        f1_sarcastic = f1_score(b_labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), average='binary', pos_label=1) 
        val_f1_sarcastic.append(f1_sarcastic)


    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    val_f1_sarcastic = np.mean(val_f1_sarcastic)

    return val_loss, val_accuracy, val_f1_sarcastic, preds_all

In [ ]:
# CNN-non-static: fastText pretrained word vectors are fine-tuned during training.
set_seed(42)
cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.25,
                                            dropout=0.25)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=20)

In [ ]:
test_loss, test_accuracy,test_f1_sarcastic,preds = evaluate(cnn_non_static, test_dataloader)

In [ ]:
test_f1_sarcastic

In [ ]:
cnn_non_static

In [ ]:
l =torch.zeros([len(preds)], dtype=torch.int32)


In [ ]:
i=0
for x in preds:
  l[i] = x.item()
  i= i+1

In [ ]:
preds = l

In [ ]:
f1_sarcastic = f1_score(test_y, l, average='binary', pos_label=1) 


In [ ]:
f1score = f1_score(test_y, preds, average='macro') 
recall = recall_score(test_y, preds, average='macro')
precision = precision_score(test_y, preds, average='macro')
report = classification_report(test_y, preds)
f1_sarcastic,f1score,recall,precision
accuracy = accuracy_score(test_y, preds)
# accuracy
# report

epoch_eval_results = {
        "accuracy":accuracy, "recall":recall, "precision":precision, "f1":f1score, "f1_sarcastic":f1_sarcastic }

In [ ]:
report = "./isarcasm/"+"_report.tsv"
sorted_report = "./isarcasm/"+"_report_sorted.tsv"
if not os.path.exists("./isarcasm/"):
  os.mkdir("./isarcasm/")


In [ ]:
report


In [ ]:
report_test = "./isarcasm/"+"_report_test.tsv"
sorted_report_test = "./isarcasm/"+"_report_test_sorted.tsv"


In [ ]:


with open(report_test,"a") as fOut:
  fOut.write(json.dumps(epoch_eval_results)+"\n")
  fOut.flush()
#------------------------------------
report_df = pd.read_json(report_test, orient='records', lines=True)
report_df.sort_values(by=["accuracy"],ascending=False, inplace=True)
report_df.to_csv(sorted_report_test,sep="\t",index=False)
report_df.head(5)

# Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

# Define Model Architecture

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask,  return_dict=False)
      # print("cls_hs", cls_hs)
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model1 = BERT_Arch(bert)

# push the model to GPU
model1 = model1.to(device)

In [ ]:
model1

In [ ]:

# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer2 = AdamW(model1.parameters(), lr = 1e-3)

# Find Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight


In [ ]:

#compute the class weights
class_wts = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y =  train_labels)

In [ ]:
print(class_wts)

In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs


# Fine-Tune BERT

In [ ]:
# function to train the model
def train2(model):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  total_labels=[]
  # iterate over batches
  for step,batch in enumerate(train_dataloader2):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader2)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer2.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)
    total_labels.append(labels)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader2)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds, total_labels

In [ ]:
# function for evaluating the model
def evaluate2(model, iterator):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  total_labels =[]
  # iterate over batches
  for step,batch in enumerate(iterator):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(iterator)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()
      labels = labels.detach().cpu().numpy()

      total_preds.append(preds)
      total_labels.append(labels)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(iterator) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds, total_labels

# Start Model Training

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
epochs=6
#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, total_pred,total_labels = train2(model1)
    
    #evaluate model
    valid_loss, val_pred,val_labels= evaluate2(model1,val_dataloader2)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model1.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

# Load Saved Model

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model1.load_state_dict(torch.load(path))

# Get Predictions for Test Data

In [ ]:
# get predictions for test data
with torch.no_grad():
  preds = model1(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

# confusion matrix
pd.crosstab(test_y, preds)

f1_sarcastic = f1_score(test_y, preds, average='binary', pos_label=1) 
f1score = f1_score(test_y, preds, average='macro') 
recall = recall_score(test_y, preds, average='macro')
precision = precision_score(test_y, preds, average='macro')
report = classification_report(test_y, preds)
print(f1_sarcastic,f1score,recall,precision)
accuracy = accuracy_score(test_y, preds)
print(accuracy)
print(report)

# Ensemble

In [ ]:
class MyEnsemble(nn.Module):

    def __init__(self, modelA, modelB, input):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        # self.modelC = modelC
        self.fc1 = nn.Linear(input, 2)

    def forward(self, sent_id, mask, b_input_ids):
        out1 = self.modelA(b_input_ids)
        out2 = self.modelB(sent_id, mask)
        out = out1 + out2 
        x = self.fc1(out)
        return torch.softmax(x, dim=1)

In [ ]:
model3 = MyEnsemble(cnn_non_static,model1, 2)

In [ ]:
# pass the pre-trained BERT to our define architecture
# model2 = BERT_Arch(bert)

# push the model to GPU
model3 = model3.to(device)

In [ ]:
optimizer3 = AdamW(model3.parameters(), lr = 1e-3)

In [ ]:
def train3(model,optimizer):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  step = 0
  for b1, b2 in zip(train_dataloader, train_dataloader2):
    # print(f, b)

  # for step,batch in enumerate(train_dataloader2):
    
    # progress update after every 50 batches.
    if step % batch_size == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader2)))
    step= step + 1

    # push the batch to gpu
    # batch = [r.to(device) for r in b1]
    b_input_ids, b_labels = tuple(t.to(device) for t in b1)
    batch = [r.to(device) for r in b2]
     
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask,b_input_ids)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer3.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate3(model,iter1,iter2):
  
  print("\nEvaluating...")
  
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  total_preds = []
  total_labels = []
  val_f1_sarcastic = []

  for b1, b2 in zip(iter1, iter2):
    # print(f, b)

 
    b_input_ids, b_labels = tuple(t.to(device) for t in b1)
    batch = [r.to(device) for r in b2]
     
    sent_id, mask, labels = batch

    with torch.no_grad():
      
      preds = model(sent_id, mask,b_input_ids)
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)
      total_labels.append(labels)

  avg_loss = total_loss / len(iter2)   
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds,total_labels

In [ ]:
best_valid_loss = float('inf')
best_val_f1_sarcastic = 0
train_losses=[]
valid_losses=[]
epochs = 1
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    train_loss, _ = train3(model3,optimizer3)
    
    valid_loss, total_preds,total_labels = evaluate3(model3,val_dataloader,val_dataloader2)
    print(total_labels)
    print(val_y)

    total_preds = np.argmax(total_preds, axis = 1)

    val_f1_sarcastic = f1_score(val_y, total_preds, average='binary', pos_label=1) 

    print("val_f1_sarcastic", val_f1_sarcastic)
    torch.save(model3.state_dict(), 'saved_weights2.pt')
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


# Load Saved Model

In [ ]:
path = 'saved_weights2.pt'
model3.load_state_dict(torch.load(path))

# Get Predictions for Test Data

In [ ]:
with torch.no_grad():
  preds = model3(test_seq2.to(device), test_mask2.to(device),torch.tensor(input_ids_test2).to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
with torch.no_grad():
  preds = model3(test_seq03.to(device), test_mask03.to(device),torch.tensor(input_ids_test03).to(device))

In [ ]:
preds = np.argmax(preds, axis = 1)

In [ ]:
pd.crosstab(test_y, preds)

In [ ]:
f1_sarcastic = f1_score(test_y, preds, average='binary', pos_label=1) 
f1score = f1_score(test_y, preds, average='macro') 
recall = recall_score(test_y, preds, average='macro')
precision = precision_score(test_y, preds, average='macro')
report = classification_report(test_y, preds)
f1_sarcastic,f1score,recall,precision
accuracy = accuracy_score(test_y, preds)
epoch_eval_results = {
        "accuracy":accuracy, "recall":recall, "precision":precision, "f1":f1score, "f1_sarcastic":f1_sarcastic }

with open(report_test,"a") as fOut:
  fOut.write(json.dumps(epoch_eval_results)+"\n")
  fOut.flush()
#------------------------------------
report_df = pd.read_json(report_test, orient='records', lines=True)
report_df.sort_values(by=["accuracy"],ascending=False, inplace=True)
report_df.to_csv(sorted_report_test,sep="\t",index=False)
report_df.head(5)

# test

In [ ]:
!cp "/gdrive/MyDrive/Master/Dataset/TestingData/task_A_AR_test.csv"  ./isarcastic/task_A_AR_test.csv

In [ ]:
def data_prepare_BERT(file_path, lab2ind, tokenizer, content_col, label_col, MAX_LEN):
  df = pd.read_csv(file_path, delimiter=',', header=0)
  df = df[df[content_col].notnull()]
  print("Data size ", df.shape)
  sentences = df[content_col].values
  sentences = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences]
  print ("The first sentence:")
  print (sentences[0])
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_texts[0])
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  print ("Index numbers of the first sentence:")
  print (input_ids[0])
  pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN+2, dtype="long", truncating="post", padding="post", value=pad_ind)
  print ("Index numbers of the first sentence after padding:\n",input_ids[0])
  attention_masks = []
  for seq in input_ids:
    seq_mask = [int(i > 0) for i in seq]
    attention_masks.append(seq_mask)
  inputs = torch.tensor(input_ids)
  masks = torch.tensor(attention_masks)
  return inputs, masks


In [ ]:
dev_file= "isarcastic/task_A_AR_test.csv" #dev file path or test file path

In [ ]:
content_col ="text"

In [ ]:
validation_inputs, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
print ("[INFO] step (6') Create an iterator of data with torch DataLoader.")

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks)


In [ ]:

#---------------------------
validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
#------------------------------------------

In [ ]:

def evaluate4(model, iterator, criterion):
  model.eval()
  epoch_loss = 0
  all_pred=[]
  with torch.no_grad():
    for i, batch in enumerate(iterator):
      batch = tuple(t.to(device) for t in batch)
      input_ids, input_mask = batch
      outputs = model(input_ids, input_mask)
      logits = outputs[0]
      del batch, input_ids, input_mask
      probabilities, predicted = torch.max(logits.cpu().data, 1)
      all_pred.extend(predicted)

  return all_pred



In [ ]:
epoch = 6

In [ ]:
test_all_pred = evaluate4(model3, validation_dataloader, criterion)

In [ ]:
for i in test_all_pred:
    print(i.item())

In [ ]:
textfile = open("task_a_ar.txt", "w")
textfile.write("task_a_ar" + "\n")
for i in preds:
    textfile.write(str(i) + "\n")
textfile.close()

In [ ]:
!cp    "task_a_ar.txt" /gdrive/MyDrive/Master/Dataset/Outputs/task_a_ar3.txt

In [ ]:
!cp "/gdrive/MyDrive/Master/Dataset/TestingData/task_C_AR_test.csv"  ./isarcastic/task_C_AR_test.csv

In [ ]:
# !cp 'MARBERT_pytorch_verison/vocab.txt' '/gdrive/MyDrive/Master/Dataset/isarcasticisarcastic_MARBERT_bert_ckpt/model_5/vocab.txt'

In [ ]:
test_file3 = 'isarcastic/task_C_AR_test.csv'

In [ ]:
test_file3 = open('isarcastic/task_C_AR_test.csv')

In [ ]:
csvreader = csv.reader(test_file3)
header = next(csvreader)
print(header)
rows = []
for row in csvreader:
    rows.append(row)
print(rows)
test_file3.close()